## CodeI/O

Original paper (DeepSeek): https://arxiv.org/pdf/2502.07316

The approach begins by obtaining high quality raw code data and preprocessing it by prompting an LLM. The output of this preprocessing, for each raw code file used, should be:

- cleaned reference code, with a main entrypoint function
- a query, converting the reference code into a question (along the lines of "given [function parameters...] how can we obtain [desired outputs...]")
- a natural language description of all inputs (function parameters) and outputs (function return values)
- an input generator, which can generate a dictionary of valid inputs for the function

This notebook seeks to experiment with prompting an LLM to this end, as a starting point. The raw code data is from this GitHub repository that the DeepSeek paper mentions as one of their raw code sources: https://github.com/TheAlgorithms/Python

In [1]:
import random
from pathlib import Path
from dotenv import load_dotenv
load_dotenv()
raw_files = list(Path("raw_files/").iterdir())

Note that the below prompt is built for DeepSeekV3. It may not work with other LLMs.

In [2]:
prompt_template = """
You are tasked with preprocessing a raw file of Python code into a standard format. The format is made up of several components. Here is a very simple example of a raw code file:

def kg_to_pounds(weights):
    return [w * 2.20462 for w in weights]

def filter_weekly(original_measurements, days):
    return [m for i, m in enumerate(original_measurements) if i % 7 == 0]

def main(kgs, days):
    lbs = kg_to_pounds(kgs)

    for measurement in filter_weekly(lbs, days):
        print(measurement)

1. Cleaned reference code, with a main entrypoint function that takes all required arguments as parameters and returns all outputs.

The name of the main entrypoint function should be `main`. The parameters should be clearly named but do not require type hints. The function should return a dict mapping output names to values. The function should contain all the necessary code to perform the functionality, without splitting into several functions. The function should not print or otherwise output anything; results should be returned as part of the result dict.

Example function signature: `def main(weights_kg, days):`

2. A query, defined as natural language description of the question the function answers.

Example query: "You are given two lists of integers, `weights_kg` and `days`. The unit of `weights_kg` is kilograms. `days` refers to the number of days passed, starting from zero. Your task is to convert the integers to pounds and filter to only one weight measurement every 7 days. Return the list of integers in pounds."

The query should be as detailed as the code requires to be fully explained. It should be clear what the function does, what the inputs are, and what the outputs are.

3. A natural language description of all inputs (function parameters) and outputs (return values) of the function.

Example description:

Input:
    weights_kg (list of int): List of weight values in kilograms.
    days (list of int): List of integers representing the number of days passed, starting from zero.

Output:
    return (dict): A dictionary with one key:
    - weights_lb (list of int): List of filtered weight values in pounds.

4. Python 3.11 code for an input generator, which randomly generates valid sets of inputs for the functions.

The input generator should return a dict mapping parameter names to values. The values should be randomly generated, but should be valid inputs for the function.

Example input generator:

def input_generator():
    weights = [np.random.uniform(0, 100) for _ in range(40)]
    days = list(range(40))
    return {{"weights_kg": weights, "days": days}}

Using the guidelines and example above, preprocess the following raw code file into the standard format:

{0}

Output the components (reference code, query, description, input generator) in order. Separate each component with a line of dashes (---). Avoid code blocks and do not output any Markdown formatting. Respond only with the four components, no prefix or additional text.
"""

In [3]:
import os
import time
from openai import OpenAI
from openai.types.chat import ChatCompletion, ChatCompletionMessageParam
from typing import Any, Iterable

def llm_generate(
    client: OpenAI,
    messages: Iterable[ChatCompletionMessageParam],
    sampling_params: dict[str, Any],
) -> ChatCompletion:
    max_retry = 3
    for trial in range(max_retry):
        try:
            return client.chat.completions.create(
                messages=messages,
                **sampling_params,
            )
        except Exception as e:
            print("failure response:", e)
            time.sleep(trial * trial)  # quadratic backoff
            if trial == max_retry - 1:
                raise

open_router_client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    timeout=90.0,
)

sampling_params = {
    "model": "deepseek/deepseek-chat:free",
    "max_tokens": 8192,
}

In [4]:
raw_file = random.choice(raw_files)

print(raw_file)

raw_code = raw_file.read_text()

prompt = prompt_template.format(raw_code)

messages = [
    {"role": "user", "content": prompt},
]

response = llm_generate(open_router_client, messages, sampling_params)
print(response.choices[0].message.content)

raw_files/bitmask.py
def main(task_performed, total_tasks):
    dp = [[-1 for _ in range(total_tasks + 1)] for _ in range(2 ** len(task_performed))]
    task = defaultdict(list)
    final_mask = (1 << len(task_performed)) - 1

    def count_ways_until(mask, task_no):
        if mask == final_mask:
            return 1
        if task_no > total_tasks:
            return 0
        if dp[mask][task_no] != -1:
            return dp[mask][task_no]

        total_ways_util = count_ways_until(mask, task_no + 1)
        for p in task[task_no]:
            if mask & (1 << p):
                continue
            total_ways_util += count_ways_until(mask | (1 << p), task_no + 1)
        
        dp[mask][task_no] = total_ways_util
        return dp[mask][task_no]

    for i in range(len(task_performed)):
        for j in task_performed[i]:
            task[j].append(i)

    total_ways = count_ways_until(0, 1)
    return {"total_ways": total_ways}

---

You are given a list `task_performed` and a

In [5]:
code, query, parameters, generator = response.choices[0].message.content.split("\n---\n")